Date Created - 6th November 2020

Date Updates - 9th November 2020

Created by - Akshay Chougule

Change log
- Create cohort for comparison

## Basics

In [38]:
import pandas as pd
import numpy as np
import requests
import datetime
import json
# from pandas.io.json import json_normalize
# import xlrd

In [39]:
import plotly.express as px 
import plotly.graph_objects as go

import dash  
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table_experiments as dt
import dash_bootstrap_components as dbc
import dash_table
import dash_daq as daq

In [40]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## AE Fetching Functions

In [41]:
def get_data(query_url):
    r = requests.get(query_url)
    j = json.loads(r.content)
    return(pd.DataFrame(j['results']))

def get_ae_pt_data(drug_brand_name):
    male_serious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:1+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    female_serious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:1+AND+patient.patientsex:0+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    male_nonserious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:2+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    female_nonserious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:2+AND+patient.patientsex:0+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    df_male_ser = get_data(male_serious_url)
    df_male_ser['gender']='male'
    df_male_ser['serious']=1
    df_female_ser = get_data(female_serious_url)
    df_female_ser['gender']='female'
    df_female_ser['serious']=1
    df_male_nonser = get_data(male_nonserious_url)
    df_male_nonser['gender']='male'
    df_male_nonser['serious']=0
    df_female_nonser = get_data(female_nonserious_url)
    df_female_nonser['gender']='female'
    df_female_nonser['serious']=0
    return(pd.concat([df_male_ser, df_female_ser,df_male_nonser,df_female_nonser], ignore_index=True))


def get_faers_data(drug_brand_name,sex,serious,meddra):
    query_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:{serious}+AND+patient.patientsex:{sex}+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddra{meddra}.exact&limit=1000'
    print(query_url)
    res_df = get_data(query_url)
    return(res_df)

def get_ae_soc_data(drug_brand_name):
    df_soc = pd.read_csv('dummy_safety_soc.csv')
    soc_copy = df_soc.copy()
    max_count = soc_copy.total_from_api.max()
    soc_copy['percent_total_api'] = round(100*(soc_copy.total_from_api/max_count),0)
    soc_copy['percent_serious_api'] = round(100*(soc_copy.serious_from_api/max_count),0)
    max_count = soc_copy.related_from_ct.max()
    soc_copy['percent_related_ct'] = round(100*(soc_copy.related_from_ct/max_count),0)
    soc_copy['percent_serious_ct'] = round(100*(soc_copy.serious_from_ct/max_count),0)
    soc_copy.sort_values('percent_total_api', ascending=False, inplace=True)
    return(soc_copy)

## Testing

In [42]:
get_faers_data("avastin",1,2,'pt')

https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:2+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000


,term,count
0,DRUG INEFFECTIVE,112
1,OFF LABEL USE,90
2,DISEASE PROGRESSION,70
3,FATIGUE,69
4,NO ADVERSE EVENT,51
...,...,...
600,VENOUS THROMBOSIS,1
601,VISUAL FIELD TESTS ABNORMAL,1
602,WHEEZING,1
603,WHITE BLOOD CELL COUNT INCREASED,1


In [43]:
df = get_faers_data('avastin',1,1,'pt')
fig = px.bar(df, x='term', y='count')
faers_tab = fig.show()

https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:1+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000


In [228]:
header = pd.MultiIndex.from_product([['Post-Marketing','Clinical-Trial'],
                                     ['Cases','Patients','Rate']],
                                    names=['','Indication'])
df = pd.DataFrame(np.random.randn(12, 6), 
                  index=["Melanoma", "Metastatic", "Non-Small Cell Lung Cancer", "Renal Cell Carcinoma", "Hodgkin's Lymphoma", "Head and Neck Cancer", "Urothelial Carcinoma", "Colorectal Cancer", "Hepatocellular Carcinoma", "Small Cell Lung Cancer", "Esophageal Carcinoma", "Malignant Pleural Mesothelioma"], 
                  columns=header)

df['Comparison','Reporting Rate']=1
df['Use-Case','Approval Status']=['Regular','Regular','Accelerated','Accelerated','Accelerated','Accelerated','Accelerated','Accelerated','Accelerated','Off-label','Off-label','Off-label']

df['Post-Marketing','Patients'] = np.random.randint(2400,high=17458, size=(12))

df['Post-Marketing','Cases'] = round(df['Post-Marketing','Patients'] * np.random.uniform(0.35,0.83,12) ,0)
df['Post-Marketing','Cases'] = df['Post-Marketing','Cases'].astype(int)

df['Post-Marketing','Rate'] = 100*round(df['Post-Marketing','Cases']/df['Post-Marketing','Patients'],2)
df['Post-Marketing','Rate'] = df['Post-Marketing','Rate'].astype(int)

df['Clinical-Trial','Patients'] = np.random.randint(800,high=2458, size=(12))

df['Clinical-Trial','Rate'] = round(df['Post-Marketing','Rate'] * np.random.uniform(0.75,1.3,12))
df['Clinical-Trial','Rate'] = df['Clinical-Trial','Rate'].astype(int)

df['Clinical-Trial','Cases'] = df['Clinical-Trial','Patients']*(df['Clinical-Trial','Rate']/100)
df['Clinical-Trial','Cases'] = df['Clinical-Trial','Cases'].astype(int)

df['Comparison','Reporting Rate']= round(df['Post-Marketing','Rate']/df['Clinical-Trial','Rate']*100,2)

In [232]:
temp = df
temp

Post-Marketing               Clinical-Trial                   Comparison        Use-Case
Indication                              Cases Patients Rate          Cases Patients Rate Reporting Rate Approval Status
Melanoma                                 2626     4858   54           1029     1775   58          93.10         Regular
Metastatic                               6978    14103   49           1378     2336   59          83.05         Regular
Non-Small Cell Lung Cancer               6868     9154   75           1811     1868   97          77.32     Accelerated
Renal Cell Carcinoma                     2439     4790   51           1396     2407   58          87.93     Accelerated
Hodgkin's Lymphoma                      13561    17124   79            816      842   97          81.44     Accelerated
Head and Neck Cancer                     2184     5586   39           1004     2009   50          78.00     Accelerated
Urothelial Carcinoma                     2025     3722   54            530     1060   50         108.00     Accelerated
Colorectal Cancer                        4331     7876   55            946     1527   62          88.71     Accelerated
Hepatocellular Carcinoma                 4905     7740   63           1553     2354   66          95.45     Accelerated
Small Cell Lung Cancer                  10239    13080   78           1348     1899   71         109.86       Off-label
Esophageal Carcinoma                     3832     6695   56            858     1908   45         124.44       Off-label
Malignant Pleural Mesothelioma           4186     9910   42            914     1905   48          87.50       Off-label

In [230]:
temp.columns.get_level_values(0)

Index(['Post-Marketing', 'Post-Marketing', 'Post-Marketing', 'Clinical-Trial', 'Clinical-Trial', 'Clinical-Trial', 'Comparison', 'Use-Case'], dtype='object', name='')

In [231]:
temp.columns.get_level_values(1)

Index(['Cases', 'Patients', 'Rate', 'Cases', 'Patients', 'Rate', 'Reporting Rate', 'Approval Status'], dtype='object', name='Indication')

In [252]:
temp2 = pd.DataFrame(temp.to_records())
temp2.columns = ['Indication','PostMarketing - Cases','PostMarketing - Patients','PostMarketing - Rate','ClinicalTrial - Cases','ClinicalTrial - Patients','ClinicalTrial - Rate', 'Reporting Rate','Usage']
temp2

,index,"('Post-Marketing', 'Cases')","('Post-Marketing', 'Patients')","('Post-Marketing', 'Rate')","('Clinical-Trial', 'Cases')","('Clinical-Trial', 'Patients')","('Clinical-Trial', 'Rate')","('Comparison', 'Reporting Rate')","('Use-Case', 'Approval Status')"
0,Melanoma,2626,4858,54,1029,1775,58,93.10,Regular
1,Metastatic,6978,14103,49,1378,2336,59,83.05,Regular
2,Non-Small Cell Lung Cancer,6868,9154,75,1811,1868,97,77.32,Accelerated
3,Renal Cell Carcinoma,2439,4790,51,1396,2407,58,87.93,Accelerated
4,Hodgkin's Lymphoma,13561,17124,79,816,842,97,81.44,Accelerated
5,Head and Neck Cancer,2184,5586,39,1004,2009,50,78.00,Accelerated
6,Urothelial Carcinoma,2025,3722,54,530,1060,50,108.00,Accelerated
7,Colorectal Cancer,4331,7876,55,946,1527,62,88.71,Accelerated
8,Hepatocellular Carcinoma,4905,7740,63,1553,2354,66,95.45,Accelerated
9,Small Cell Lung Cancer,10239,13080,78,1348,1899,71,109.86,Off-label


In [253]:
#temp2.reset_index(inplace=True)


,Indication,PostMarketing - Cases,PostMarketing - Patients,PostMarketing - Rate,ClinicalTrial - Cases,ClinicalTrial - Patients,ClinicalTrial - Rate,Reporting Rate,Usage
0,Melanoma,2626,4858,54,1029,1775,58,93.10,Regular
1,Metastatic,6978,14103,49,1378,2336,59,83.05,Regular
2,Non-Small Cell Lung Cancer,6868,9154,75,1811,1868,97,77.32,Accelerated
3,Renal Cell Carcinoma,2439,4790,51,1396,2407,58,87.93,Accelerated
4,Hodgkin's Lymphoma,13561,17124,79,816,842,97,81.44,Accelerated
5,Head and Neck Cancer,2184,5586,39,1004,2009,50,78.00,Accelerated
6,Urothelial Carcinoma,2025,3722,54,530,1060,50,108.00,Accelerated
7,Colorectal Cancer,4331,7876,55,946,1527,62,88.71,Accelerated
8,Hepatocellular Carcinoma,4905,7740,63,1553,2354,66,95.45,Accelerated
9,Small Cell Lung Cancer,10239,13080,78,1348,1899,71,109.86,Off-label


In [224]:
temp.columns = temp.columns.get_level_values(0)
temp.columns = [''.join(col).strip() for col in temp.columns.values]

In [225]:
temp

,Post-Marketing,Post-Marketing,Post-Marketing,Clinical-Trial,Clinical-Trial,Clinical-Trial,Comparison,Use-Case
Melanoma,8206,13130,62,758,1244,61,101.64,Regular
Metastatic,1970,3631,54,1080,2160,50,108.00,Regular
Non-Small Cell Lung Cancer,4129,7426,56,999,1998,50,112.00,Accelerated
Renal Cell Carcinoma,5200,13879,37,578,1754,33,112.12,Accelerated
Hodgkin's Lymphoma,3432,6517,53,881,1574,56,94.64,Accelerated
Head and Neck Cancer,7758,12351,63,809,1285,63,100.00,Accelerated
Urothelial Carcinoma,5355,15023,36,439,1256,35,102.86,Accelerated
Colorectal Cancer,6031,13867,43,777,1654,47,91.49,Accelerated
Hepatocellular Carcinoma,10244,16111,64,545,957,57,112.28,Accelerated
Small Cell Lung Cancer,1045,2761,38,1127,2399,47,80.85,Off-label


Post-Marketing               Clinical-Trial                   Comparison        Use-Case
Indication                              Cases Patients Rate          Cases Patients Rate Reporting Rate Approval Status
Melanoma                                10161    12269   83            767      936   82         101.22         Regular
Metastatic                               9042    13678   66           1219     2217   55         120.00         Regular
Non-Small Cell Lung Cancer               5137     6403   80           1200     2001   60         133.33     Accelerated
Renal Cell Carcinoma                    11694    14269   82           1487     1566   95          86.32     Accelerated
Hodgkin's Lymphoma                       3187     6890   46            928     2264   41         112.20     Accelerated
Head and Neck Cancer                    12002    17148   70           1841     2166   85          82.35     Accelerated
Urothelial Carcinoma                     2485     5668   44            820     1465   56          78.57     Accelerated
Colorectal Cancer                        8547    11657   73            975     1549   63         115.87     Accelerated
Hepatocellular Carcinoma                10430    14712   71            846      962   88          80.68     Accelerated
Small Cell Lung Cancer                   3680     5796   63           1197     1478   81          77.78       Off-label
Esophageal Carcinoma                     2589     4432   57            963     1357   71          80.28       Off-label
Malignant Pleural Mesothelioma           8820    11048   80           1482     2246   66         121.21       Off-label

In [206]:
temp['indication',''] = temp.index
temp

Post-Marketing               Clinical-Trial                   Comparison        Use-Case                      indication
Indication                              Cases Patients Rate          Cases Patients Rate Reporting Rate Approval Status                                
Melanoma                                10161    12269   83            767      936   82         101.22         Regular                        Melanoma
Metastatic                               9042    13678   66           1219     2217   55         120.00         Regular                      Metastatic
Non-Small Cell Lung Cancer               5137     6403   80           1200     2001   60         133.33     Accelerated      Non-Small Cell Lung Cancer
Renal Cell Carcinoma                    11694    14269   82           1487     1566   95          86.32     Accelerated            Renal Cell Carcinoma
Hodgkin's Lymphoma                       3187     6890   46            928     2264   41         112.20     Accelerated              Hodgkin's Lymphoma
Head and Neck Cancer                    12002    17148   70           1841     2166   85          82.35     Accelerated            Head and Neck Cancer
Urothelial Carcinoma                     2485     5668   44            820     1465   56          78.57     Accelerated            Urothelial Carcinoma
Colorectal Cancer                        8547    11657   73            975     1549   63         115.87     Accelerated               Colorectal Cancer
Hepatocellular Carcinoma                10430    14712   71            846      962   88          80.68     Accelerated        Hepatocellular Carcinoma
Small Cell Lung Cancer                   3680     5796   63           1197     1478   81          77.78       Off-label          Small Cell Lung Cancer
Esophageal Carcinoma                     2589     4432   57            963     1357   71          80.28       Off-label            Esophageal Carcinoma
Malignant Pleural Mesothelioma           8820    11048   80           1482     2246   66         121.21       Off-label  Malignant Pleural Mesothelioma

In [198]:
temp = temp.reset_index()
temp

index Post-Marketing               Clinical-Trial                   Comparison        Use-Case
Indication                                          Cases Patients Rate          Cases Patients Rate Reporting Rate Approval Status
0                                 Melanoma          10161    12269   83            767      936   82         101.22         Regular
1                               Metastatic           9042    13678   66           1219     2217   55         120.00         Regular
2               Non-Small Cell Lung Cancer           5137     6403   80           1200     2001   60         133.33     Accelerated
3                     Renal Cell Carcinoma          11694    14269   82           1487     1566   95          86.32     Accelerated
4                       Hodgkin's Lymphoma           3187     6890   46            928     2264   41         112.20     Accelerated
5                     Head and Neck Cancer          12002    17148   70           1841     2166   85          82.35     Accelerated
6                     Urothelial Carcinoma           2485     5668   44            820     1465   56          78.57     Accelerated
7                        Colorectal Cancer           8547    11657   73            975     1549   63         115.87     Accelerated
8                 Hepatocellular Carcinoma          10430    14712   71            846      962   88          80.68     Accelerated
9                   Small Cell Lung Cancer           3680     5796   63           1197     1478   81          77.78       Off-label
10                    Esophageal Carcinoma           2589     4432   57            963     1357   71          80.28       Off-label
11          Malignant Pleural Mesothelioma           8820    11048   80           1482     2246   66         121.21       Off-label

In [201]:
temp.columns

MultiIndex([(         'index',                ''),
            ('Post-Marketing',           'Cases'),
            ('Post-Marketing',        'Patients'),
            ('Post-Marketing',            'Rate'),
            ('Clinical-Trial',           'Cases'),
            ('Clinical-Trial',        'Patients'),
            ('Clinical-Trial',            'Rate'),
            (    'Comparison',  'Reporting Rate'),
            (      'Use-Case', 'Approval Status')],
           names=['', 'Indication'])

In [203]:
temp.columns==['index']

array([False, False, False, False, False, False, False, False, False])

In [ ]:
df.columns.rename("index", level=1)

In [101]:
np.random.rand(12)

array([0.81264935, 0.19755237, 0.95606384, 0.57655822, 0.78270243,
       0.0970228 , 0.95345794, 0.98955604, 0.72721614, 0.40646026,
       0.19659982, 0.77476786])

In [143]:
np.random.uniform(0.35,1.3,12) 

array([1.0371443 , 0.57722831, 0.71140695, 1.00164932, 0.75449022,
       0.47902622, 1.1013188 , 0.88011523, 0.59590799, 0.54600258,
       1.06056006, 1.05456266])

In [72]:
header = pd.MultiIndex.from_product([['Post-Marketing','Clinical-Trial','Comparison'],
                                     ['Cases','Patient','Rates']],
                                    names=['loc','S'])
df = pd.DataFrame(np.random.randn(5, 9), 
                  index=['a','b','c','d','e'], 
                  columns=header)
df

loc Post-Marketing                     Clinical-Trial                     Comparison                    
S            Cases   Patient     Rates          Cases   Patient     Rates      Cases   Patient     Rates
a        -0.968244 -0.729901 -0.048225      -0.313224  1.282767  0.633464   0.939201 -0.598703 -1.145199
b         0.064320 -0.345105 -0.627148      -0.627585  0.200777 -2.709252   0.501639  0.282120 -0.599267
c        -1.455255  2.122893 -0.716653      -0.630364 -0.694878 -0.260124   0.384643  1.514508  1.211407
d        -0.944828 -1.181708  1.075183       0.884814  1.500320 -2.131665  -1.289243  0.815804 -0.418095
e        -0.522170  1.333894 -0.560233       0.214904 -1.337839  1.760611  -0.679590  0.220324  0.415977

## Dash App

In [44]:
tabs_styles = {
    'height': '51px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '2px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '5px solid orange',
    'backgroundColor': '#4c2d4dff',
    'color': 'white',
    'padding': '10px'
}

### Quick Summary

In [61]:
quick_summary = html.Div([
            html.Br(),html.Br(),
            dbc.Row([
                dbc.Col(dbc.Card([
                    dbc.CardHeader("FAERS"),
                    dbc.CardBody(
                        [
                            html.P("Cardiac disorders"),
                            html.P("Reproductive system and breast disorders"),
                            html.P("Gastrointestinal disorders"),
                            html.P("Blood and lymphatic system disorders"),   
                            html.P("Hepatobiliary disorders"),
                            dbc.Button("explore more", color="secondary")
                        ]
                    )]
                )),

                dbc.Col(dbc.Card([
                    dbc.CardHeader("EudraVigilance"),
                    dbc.CardBody(
                        [
                            html.P("Gastrointestinal disorders"),
                            html.P("Blood and lymphatic system disorders"),
                            html.P("Cardiac disorders"),
                            html.P("Reproductive system and breast disorders"),
                            html.P("Hepatobiliary disorders"),
                            dbc.Button("explore more", color="secondary")
                        ]
                    )
                ])),

                dbc.Col(dbc.Card([
                    dbc.CardHeader("VigiBase"),
                    dbc.CardBody(
                        [
                            html.P("Blood and lymphatic system disorders"),
                            html.P("Gastrointestinal disorders"),
                            html.P("Cardiac disorders"),
                            html.P("Reproductive system and breast disorders"),
                            html.P("Hepatobiliary disorders"),
                            dbc.Button("explore more", color="secondary")
                        ]
                    )
                ])),
                
                dbc.Col(dbc.Card([
                    dbc.CardHeader("Clinical Trial"),
                    dbc.CardBody(
                        [
                            html.P("Gastrointestinal disorders"),
                            html.P("Blood and lymphatic system disorders"),
                            html.P("Cardiac disorders"),
                            html.P("Reproductive system and breast disorders"),
                            html.P("Hepatobiliary disorders"),
                            dbc.Button("explore more", color="secondary"),
                        ]
                    )
                ])),
            ])
        ])

### Drug Event Analysis

In [288]:
header = pd.MultiIndex.from_product([['Post-Marketing','Clinical-Trial'],
                                     ['Cases','Patients','Rate']],
                                    names=['','Indication'])
df = pd.DataFrame(np.random.randn(12, 6), 
                  index=["Melanoma", "Metastatic", "Non-Small Cell Lung Cancer", "Renal Cell Carcinoma", "Hodgkin's Lymphoma", "Head and Neck Cancer", "Urothelial Carcinoma", "Colorectal Cancer", "Hepatocellular Carcinoma", "Small Cell Lung Cancer", "Esophageal Carcinoma", "Malignant Pleural Mesothelioma"], 
                  columns=header)

df['Comparison','Reporting Rate']=1
df['Use-Case','Approval Status']=['Regular','Regular','Accelerated','Accelerated','Accelerated','Accelerated','Accelerated','Accelerated','Accelerated','Off-label','Off-label','Off-label']

df['Post-Marketing','Patients'] = np.random.randint(2400,high=17458, size=(12))

df['Post-Marketing','Cases'] = round(df['Post-Marketing','Patients'] * np.random.uniform(0.35,0.83,12) ,0)
df['Post-Marketing','Cases'] = df['Post-Marketing','Cases'].astype(int)

df['Post-Marketing','Rate'] = 100*round(df['Post-Marketing','Cases']/df['Post-Marketing','Patients'],2)
df['Post-Marketing','Rate'] = df['Post-Marketing','Rate'].astype(int)

df['Clinical-Trial','Patients'] = np.random.randint(800,high=2458, size=(12))

df['Clinical-Trial','Rate'] = round(df['Post-Marketing','Rate'] * np.random.uniform(0.75,1.3,12))
df['Clinical-Trial','Rate'] = df['Clinical-Trial','Rate'].astype(int)

df['Clinical-Trial','Cases'] = df['Clinical-Trial','Patients']*(df['Clinical-Trial','Rate']/100)
df['Clinical-Trial','Cases'] = df['Clinical-Trial','Cases'].astype(int)

df['Comparison','Reporting Rate']= round(df['Post-Marketing','Rate']/df['Clinical-Trial','Rate']*100,2)

temp = pd.DataFrame(df.to_records())
temp.columns = ['indication','pm_cases','pm_patients','pm_rate','ct_cases','ct_patients','ct_rate', 'compare','usecase']
#temp

drug_event_data = dash_table.DataTable(
                        id='table',
#                         columns=[{"name": i, "id": i} for i in temp.columns],
                        columns=[
                            {"name": ["", "Indication"], "id": "indication"},
                            {"name": ["Post-Marketing", "Cases"], "id": "pm_cases"},
                            {"name": ["Post-Marketing", "Patients"], "id": "pm_patients"},
                            {"name": ["Post-Marketing", "Rate"], "id": "pm_rate"},
                            {"name": ["Clinical-Trial", "Cases"], "id": "ct_cases"},
                            {"name": ["Clinical-Trial", "Patients"], "id": "ct_patients"},
                            {"name": ["Clinical-Trial", "Rate"], "id": "ct_rate"},
                            {"name": ["Comparison", "Reporting Rate"], "id": "compare"},
                            {"name": ["", "Use-Case"], "id": "usecase"},
                        ],
                        data=temp.to_dict("rows"),
                        merge_duplicate_headers=True,
                        filter_action="native",
                        sort_action="native",
                        #export_format="csv",
                        #export_headers="display",
                        style_cell={'width': '150px',
                        'height': '40px',
                        'textAlign': 'left'},
                        style_data_conditional=[{
#                             'if': {'row_index': 'odd'},
#                             'backgroundColor': 'rgb(248, 248, 248)',
                            'if': {
                                        #'filter_query': '{{compare}} = {}'.format(temp['compare'].max()),
                                        'filter_query': '{compare} >= 100',
                                        'column_id': 'compare'
                                    },
                                    'backgroundColor': '#FF4136',
                                    'color': 'white'
                        }],
                        style_header={
                            'backgroundColor': 'rgb(230, 230, 230)',
                            'fontWeight': 'bold'
                        })

drug_event=html.Div([html.Br(),
                     html.Div([], style={'width': '2%', 'display': 'inline-block'}),
                     html.Div(drug_event_data, style={'width': '96%', 'display': 'inline-block'}),
                     html.Div([], style={'width': '2%', 'display': 'inline-block'}),
                    ])

In [185]:
#drug_event_data

### Input Components

In [67]:
race = dbc.FormGroup([
                dbc.Label("Race", html_for="dropdown"),
                dcc.Dropdown(id="select_race",
                 options=[
                     {"label": "White", "value": "white"},
                     {"label": "Black", "value": "black"},
                     {"label": "Asian", "value": "asian"},
                     {"label": "Pacific Islander", "value": "islander"},
                     {"label": "Other or Unknown", "value": "other"},
                     ],
                     multi=True
                             ),
])

sex = dbc.FormGroup([
                dbc.Label("Sex", html_for="dropdown"),
                dcc.Dropdown(id="select_sex",
                 options=[
                     {"label": "Any", "value": "all"},
                     {"label": "Male", "value": "male"},
                     {"label": "Female", "value": "female"},
                     ],
                    value='male'),
])


drug = dbc.FormGroup([
                dbc.Label("Drug Name", html_for="dropdown"),
                dcc.Dropdown(id="select_drug",
                 options=[
                     {"label": "Avastin", "value": "avastin"},
                     {"label": "Cyramza", "value": "cyramza"},
                     {"label": "Humira", "value": "humira"},
                     {"label": "Opdiva", "value": "opdiva"},
                     ],
                    value='avastin'),
])


drug_class = dbc.FormGroup([
                dbc.Label("Drug Name or Class", html_for="dropdown"),
                dcc.Dropdown(id="select_drugclass",
                 options=[
                     {"label": "Avastin", "value": "avastin"},
                     {"label": "Humira", "value": "humira"},
                     {"label": "Cyramza", "value": "cyramza"},
                     ],
                    value='avastin'),
])

data_sources = dbc.FormGroup([
                dbc.Label("Data Sources", html_for="dropdown"),
                dcc.Dropdown(id="select_source",
                 options=[
                     {"label": "FAERS (FDA)", "value": "faers"},
                     {"label": "Clinical Trials", "value": "trials"},
                     {"label": "EMR Data (RWD)", "value": "rwd"},
                     {"label": "EudraVigilance (EMA)", "value": "eudravigilance"},
                     {"label": "VigiBase (WHO)", "value": "vigibase"},
                     ],
                    value='avastin'),
])

serious = dbc.FormGroup([
                dbc.Label("Serious", html_for="dropdown"),
                dcc.Dropdown(id="select_serious",
                 options=[
                     {"label": "Serious", "value": "serious"},
                     {"label": "Non-serious", "value": "nonserious"},
                     ],
                     value='serious'
                     #,multi=True
                             ),
])


meddra = dbc.FormGroup([
                dbc.Label("MedDRA Level", html_for="dropdown"),
                dcc.Dropdown(id="select_meddra",
                 options=[
                     {"label": "System Organ Class (SOC)", "value": "soc"},
                     {"label": "Preferred Term (PT)", "value": "pt"},
                     ],
                     value='soc'
                     #,multi=True
                             ),
])

data_switch = dbc.FormGroup([
                dbc.Label("Data Table to Viz"),
                daq.ToggleSwitch(
                    id='view_data',
                    color='#4c2d4dff',
                    value=False
                )
])

meddra_switch = dbc.FormGroup([
                dbc.Label("SOC to PT"),
                daq.ToggleSwitch(
                    id='view_pt',
                    color='#4c2d4dff',
                    value=False
                )
])


date_slider = dcc.DatePickerRange(
    start_date_placeholder_text="Start Period",
    end_date_placeholder_text="End Period",
    calendar_orientation='vertical',
)

analysis = dbc.FormGroup([
                dcc.Dropdown(id="select_analysis",
                 options=[
                     {"label": "Quick Summary (Top N)", "value": "top_n"},
                     {"label": "Demographic profile", "value": "demog"},
                     {"label": "Drug-Event", "value": "drug_event"},
                     {"label": "Indication-Event", "value": "indi_event"},
                     {"label": "Compare Product", "value": "comp_prod"},
                     {"label": "Compare Class", "value": "comp_class"},
                     ],
                     value='top_n'
                     #,multi=True
                             ),
])


### App Layout

In [291]:
app = dash.Dash(external_stylesheets = [dbc.themes.BOOTSTRAP])

# App layout
app.layout = html.Div([
    
    html.H1("Safety Data Explorer", style={'text-align': 'center',
                                           'padding': '4px',
                                           'color': '#4c2d4dff'}),
    
    html.H5("Based on dummy data", style={'text-align': 'center',
                                           'padding': '2px',
                                           'color': '#4c2d4dff'}),
    
    # Div for all tabs   
    html.Div(dcc.Tabs([
        
        ### Tab 1 ###
        dcc.Tab(label='Explore Safety Data', children=[
            
            ### Tab 1 - col 1 ###
            html.Div([], style={'width': '1%', 'display': 'inline-block'}),
            
            html.Div([html.Br(),
                    drug, 
                    html.H5("Select filters", style={'text-align': 'center','color': '#4c2d4dff'}),
                    data_sources, serious, sex, race,
                    dbc.Label("Date Range", html_for="range-slider"),
                    date_slider, html.Br(),
                    html.Div([html.H5("Select Analysis", style={'text-align': 'center','color': '#4c2d4dff'})], style={'marginBottom': '1em', 'marginTop': '1em'}),
                    analysis, 
                    html.H5("Change View", style={'text-align': 'center','color': '#4c2d4dff'}),
                    data_switch, meddra_switch
            ], style={'width': '12%', 'display': 'inline-block'}),
            
            ### Tab 1 - col 2 ###
            html.Div([], style={'width': '2%', 'display': 'inline-block'}),
            
            ### Tab 1 - col 3 ###
            html.Div(id='exploration-output', style={'width': '85%', 'display': 'inline-block', 'vertical-align': 'top'}),                       
        ], style=tab_style, selected_style=tab_selected_style),
        
        
        ### Tab 2 ###
        dcc.Tab(label='Custom Analysis', children=[
            ### Tab 2 - col 1 ###
            #html.Div(tt, style={'width': '13%', 'display': 'inline-block'}),          
            
            ### Tab 2 - col 2 ###
            html.Div(id='compare-gap', style={'width': '16%', 'display': 'inline-block'}),
            
            ### Tab 2 - col 3 ###
            
            ### Tab 2 - col 4 ###
            html.Div(id='compare-output', style={'width': '84%', 'display': 'inline-block'}),
            
     ], style=tab_style, selected_style=tab_selected_style),

    ])) # Div for all tabs end here
    
])


# ------------------------------------------------------------------------------
# Connect the data with Dash Components
@app.callback(
    dash.dependencies.Output('exploration-output', 'children'),
    [Input(component_id='select_drug', component_property='value'),
     Input(component_id='select_source', component_property='value'),
     Input(component_id='select_sex', component_property='value'), 
     Input(component_id='select_serious', component_property='value'),
     Input(component_id='select_analysis', component_property='value'),
     Input(component_id='view_data', component_property='value'),
     Input(component_id='view_pt', component_property='value')]
)


# update logic
def update_graph(drug_name, select_source, select_sex, select_serious, select_analysis, view_data, view_pt):
    print(drug_name, select_source, select_sex, select_serious, select_analysis, view_data, view_pt)  
    
    out = html.Div([
        html.H5("Select the filters and type of analysis"),
    ])
    
    ############################
    ###   top n analysis     ###
    ############################
    if select_analysis=='top_n':
        out = quick_summary
    
    ############################
    ###   Demog analysis     ###
    ############################
    if select_analysis=='demog':
        print('demog')
        out = html.Div([
            html.H5("Select the filters and type of analysis"),
        ])
    
    ############################
    ### drug-event analysis  ###
    ############################
    if select_analysis=='drug_event':
        print('demog')
        out = drug_event
        
    ############################
    ### indi-event analysis  ###
    ############################
    if select_analysis=='indi_event':
        print('demog')
        out = html.Div([
            html.H5("Select the filters and type of analysis"),
        ])
        
    ############################
    ###   compare product    ###
    ############################
    if select_analysis=='comp_prod':
        print('demog')
        out = html.Div([
            html.H5("Select the filters and type of analysis"),
        ])
    
    ############################
    ###   compare class      ###
    ############################
    if select_analysis=='comp_class':
        print('demog')
        out = html.Div([
            html.H5("Select the filters and type of analysis"),
        ])
        

    return out
    


# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Nov/2020 23:28:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2020 23:28:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2020 23:28:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2020 23:28:25] "POST /_dash-update-component HTTP/1.1" 200 -


avastin avastin male serious top_n False False


### app run output

In [21]:
drug_class1 = dbc.FormGroup([
                dbc.Label("Drug Name or Class", html_for="dropdown"),
                dcc.Dropdown(id="select_drugclass_c1",
                 options=[
                     {"label": "Avastin", "value": "avastin"},
                     {"label": "Docetaxel", "value": "docetaxel"},
                     {"label": "Carboplatin", "value": "carboplatin"},
                     ],
                    value='avastin'),
])

data_sources1 = dbc.FormGroup([
                dbc.Label("Data Sources", html_for="dropdown"),
                dcc.Dropdown(id="select_source_c1",
                 options=[
                     {"label": "FAERS (FDA)", "value": "faers"},
                     {"label": "Clinical Trials", "value": "trials"},
                     {"label": "EMR Data (RWD)", "value": "rwd"},
                     {"label": "EudraVigilance (EMA)", "value": "eudravigilance"},
                     {"label": "VigiBase (WHO)", "value": "vigibase"},
                     ],
                    value='avastin'),
])

serious_c1 = dbc.FormGroup([
                dbc.Label("Serious c1", html_for="dropdown"),
                dcc.Dropdown(id="select_serious_c1",
                 options=[
                     {"label": "Serious", "value": "serious"},
                     {"label": "Non-serious", "value": "nonserious"},
                     ],
                     value='serious'
                     #,multi=True
                             ),
])


meddra_c1 = dbc.FormGroup([
                dbc.Label("MedDRA Level c1", html_for="dropdown"),
                dcc.Dropdown(id="select_meddra_c1",
                options=[
                    {"label": "System Organ Class (SOC)", "value": "soc"},
                    {"label": "Preferred Term (PT)", "value": "pt"},
                    ],
                    value='soc'
                            )
]),

sex_c1 = dbc.FormGroup([
                dbc.Label("Sex c1", html_for="dropdown"),
                dcc.Dropdown(id="select_sex_c1",
                 options=[
                     {"label": "Any", "value": "all"},
                     {"label": "Male", "value": "male"},
                     {"label": "Female", "value": "female"},
                     ],
                    value='male'),
]),
    
drug_class2 = dbc.FormGroup([
                dbc.Label("Drug Name or Class c2", html_for="dropdown"),
                dcc.Dropdown(id="select_drugclass_c2",
                 options=[
                     {"label": "Avastin", "value": "avastin"},
                     {"label": "Docetaxel", "value": "docetaxel"},
                     {"label": "Carboplatin", "value": "carboplatin"},
                     ],
                    value='avastin'),
]),

data_sources2 = dbc.FormGroup([
                dbc.Label("Data Sources c2", html_for="dropdown"),
                dcc.Dropdown(id="select_source_c2",
                 options=[
                     {"label": "FAERS (FDA)", "value": "faers"},
                     {"label": "Clinical Trials", "value": "trials"},
                     {"label": "EMR Data (RWD)", "value": "rwd"},
                     {"label": "EudraVigilance (EMA)", "value": "eudravigilance"},
                     {"label": "VigiBase (WHO)", "value": "vigibase"},
                     ],
                    value='avastin'),
])

serious2 = dbc.FormGroup([
                dbc.Label("Serious c2", html_for="dropdown"),
                dcc.Dropdown(id="select_serious_c2",
                 options=[
                     {"label": "Serious", "value": "serious"},
                     {"label": "Non-serious", "value": "nonserious"},
                     ],
                     value='serious'
                     #,multi=True
                             ),
])


meddra2 = dbc.FormGroup([
                dbc.Label("MedDRA Level c2", html_for="dropdown"),
                dcc.Dropdown(id="select_meddra_c2",
                 options=[
                     {"label": "System Organ Class (SOC)", "value": "soc"},
                     {"label": "Preferred Term (PT)", "value": "pt"},
                     ],
                     value='soc'
                             ),
])

sex2 = dbc.FormGroup([
                dbc.Label("Sex c2", html_for="dropdown"),
                dcc.Dropdown(id="select_sex_c2",
                 options=[
                     {"label": "Any", "value": "all"},
                     {"label": "Male", "value": "male"},
                     {"label": "Female", "value": "female"},
                     ],
                    value='male'),
]),